In [1387]:
import pandas as pd
import numpy as np
import math
import re

## Загрузка датасета

In [2]:
ds = pd.read_csv("hillary-clinton-emails/Emails.csv", 
                 delimiter=",", usecols =["MetadataSubject","ExtractedSubject", "ExtractedBodyText","RawText"])

## Предобработка датасета

В качестве текста исходных писем выбиралось поле ExtractedBodyText. 

In [3816]:
ds.ExtractedBodyText[20]

"H <hrod17@clintonernaii.com›\nWednesday, September 12, 2012 11:26 PM\nesullivanjj@state.gov'\nFw: Fwd: more on libya\nLibya 37 sept 12 12,docx\nWe should get this around asap."

На основе "плохих" частотных слов и структуры исходных писем выделила небольшой список стоп-слов и выражений, которые надо удалить из исходного текста перед обработкой. Например, строки с датой и пр.метаинформация.

In [3979]:
stop_words = ["UNCLASSIFIED", "U.S. Department of State","hillary"]
stop_lines = ["case no", "doc no", "importance","cc:", "Cc","Sent from", "h <hrod17@clintonemail.com>", 
              "sunday", "monday","tuesday", "wednesday", "thursday","friday", "saturday"]
stop_phrases = ["@state.gov","@clintonemail","©state.gov"]

Удаляю стоп-слова из писем. Слишком короткие письма не включаю в выборку.

In [3980]:
mails = []
titles = []
for i in range(1, len(ds.index)):
    cur_mail = ""
  
    if type(ds.ExtractedBodyText[i]) is str and len(ds.ExtractedBodyText[i]) > 0:
        lines = ds.ExtractedBodyText[i].split("\n")
        cur_mail = ""
        for line in lines:
            for sl in stop_lines:
                if line.lower().startswith(sl):
                    line = ""
                    break
            for sp in stop_phrases:
                if line.find(sp) > -1:
                    line = ""
                    break
            if re.search('.docx', line) != None:
                line = ""
                
            if len(line) != 0:
                cur_word = line.replace("\n", " ")
                for sw in stop_words:
                    cur_word = cur_word.replace(sw, "")
                cur_mail += " " + cur_word + "."
    if len(cur_mail) > 20:
        mails.append(cur_mail)
    if type(ds.ExtractedSubject[i]) is str and len(ds.ExtractedSubject[i]) > 3:
        titles.append(ds.ExtractedSubject[i].replace("\n", " ") +". ")

### Токенизация

На этом этапе удаляю слишком короткие слова(скорее всего они большой смысловой нагрузки не несут) + использую стеммер

In [3981]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [3982]:
stemmer = PorterStemmer()
stemToWord = {}
def tokenize_and_stem(text):
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    for token in tokens:
        if len(token)>3 and re.match('[A-Za-z]+', token) != None:
            filtered_tokens.append(token.strip("."))
    stems = []
    tags = nltk.pos_tag(filtered_tokens)
    for token, tag in tags:
        stem = stemmer.stem(token)
        if len(stem)>3 and (tag[0] == "N" or tag[0]=="V"):
            stems.append(stem)
            stemToWord[stem]=token
    return stems

In [3983]:
tokenize_and_stem(mails[0])

['latest', 'syria', 'qaddafi', 'march', 'hillari']

In [3984]:
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict

def filterFrequentWords(n, percent, mails):
    vectorizer = CountVectorizer(ngram_range=(n,n),tokenizer=tokenize_and_stem, stop_words=stopwords.words("english"))
    X = vectorizer.fit_transform(mails)
    
    ngramms = defaultdict(int)
    END_OF_SENTENCE = "<END>"
    feature_names = vectorizer.get_feature_names()
    count = [len(np.nonzero(X[0,:])[1]) ]
    wordsCount = 0
    for i in X.nonzero()[1]:
        ngramms[feature_names[i]] += 1
        wordsCount+=1
    
    countToToken = defaultdict(list)
    for key,value in ngramms.items():
        countToToken[value].append(key)
    maxCount = sorted(list(countToToken), reverse=True)
    frequentTokens = [] 
    for c in maxCount:
        frequentTokens += countToToken[c] 
    return frequentTokens[0:int(wordsCount*percent)]
    

## Биграммы

In [3985]:
BiGramms = filterFrequentWords(2, 1, mails)

## Коллокации

In [3986]:
from nltk.collocations import *

In [3987]:
from nltk.tokenize import word_tokenize, sent_tokenize, TweetTokenizer
words = []
END_OF_SENTENCE = "<END>"

for mail in mails:
    words.extend(word_tokenize(mail))
    words.append(END_OF_SENTENCE)

In [3988]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(words)
finder.nbest(bigram_measures.pmi, 10) 

[("'1.com", 'aeirig'),
 ("'1HE", ".i't"),
 ("'Banish", 'Sexual'),
 ("'H", 'CCL'),
 ("'Li", 'Matou'),
 ("'Stone", 'Harbour'),
 ("'bad", 'talkers'),
 ("'civ", 'cas'),
 ("'ffrjend", 'nieei'),
 ("'i", 'Voz')]

In [4079]:

def countNGramms(n, mails, indexes):
    model = defaultdict(int)
    
    END_OF_SENTENCE = "<END>"
    for id in indexes:
        words = tokenize_and_stem(mails[id])
            
        for i in range(n-1, len(words)):
            ngramma = "_".join(words[i - (n-1):i+1])
            model[ngramma] += 1
    countToModel = defaultdict(list)
    for key, value in model.items():
        countToModel[value].append(key)
    sortedCount = sorted(countToModel.keys(),reverse=True)
    results = []
    for c in sortedCount[:20]:
        results.extend(countToModel[c])
    return results[:10]
    

## Обучение

Ищу потенциальные стоп-слова - слова, которые встречаются в большом количестве документов(высокая df). Параметром max_df не пользовалась, потому что хотелось посмотреть на частотные слова.

In [4018]:
stopUniGramms = filterFrequentWords(1, 0.0005, mails)


In [4019]:
print(len(stopUniGramms))
print(stopUniGramms)

62
['call', 'work', 'state', 'time', 'want', 'know', 'meet', 'secretari', 'talk', 'today', 'need', 'think', 'make', 'offic', 'tomorrow', 'presid', 'issu', 'come', 'take', 'said', 'week', 'thank', 'year', 'hous', 'discuss', 'peopl', 'help', 'part', 'secur', 'report', 'sent', 'includ', 'govern', 'point', 'follow', 'support', 'hope', 'send', 'look', 'countri', 'plan', 'polici', 'made', 'clinton', 'world', 'thing', 'email', 'releas', 'depart', 'chang', 'washington', 'leader', 'give', 'morn', 'minist', 'speech', 'group', 'assist', 'told', 'statement', 'staff', 'messag']


In [4020]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(ngram_range=(1,2),max_df=0.1, min_df = 2,tokenizer=tokenize_and_stem, 
                             stop_words=stopwords.words("english")+stopUniGramms)
X = vectorizer.fit_transform(mails)

In [4021]:

feature_names = vectorizer.get_feature_names()

In [4022]:
len(feature_names)

19013

In [4073]:
n_clusters=7
from sklearn.cluster import KMeans
km = KMeans(n_clusters=n_clusters, random_state=20)


In [4074]:
km.fit(X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=7, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=20, tol=0.0001, verbose=0)

## Результаты

Количество писем для каждого кластера. При любых экспериментах выделяется крупный кластер и несколько маленьких. 
Маленькие кластеры характеризуются редкими словами. Большой описывается более частотными.

In [4075]:
km.labels_.sort()
curIndex = 0
curCount = 0
for i in range(len(km.labels_)):
    if km.labels_[i] != curIndex:
        print(curIndex, curCount)
        curCount = 0
        curIndex += 1

    curCount += 1
print(curIndex, curCount)
    

0 179
1 103
2 61
3 76
4 4157
5 634
6 114


Нашла примеры использования описания центра кластера в качестве характеристики. 

Темы угадываются, но если центр удален, может быть не очень хорошо. Но набор тем определить можно.

Вообще кажется, что на очень частотные слова смотреть плохо, потому что кластеры могут быть вытянуты по каким-то напрвлениям.

In [4076]:
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

for i in range(n_clusters):
    print("Cluster ", i, " centroid")
    print(", ".join([
            " ".join([stemToWord[name] for name in feature_names[ind].split(" ")])
                for ind in order_centroids[i, :10]]))
    print()
    print() 

Cluster  0  centroid
Huma, Lona, Abedin, Valmoro, sheet, Lona Valmoro, Abedin Huma, Huma Abedin, check, Jake


Cluster  1  centroid
print, print copy, Oscar print, Oscar, copy, Huma print, Huma, print Review, please print, Review


Cluster  2  centroid
date, BENGHAZI, REDACTIONS, FOIA, agreement information, REDACTIONS FOIA, information REDACTIONS, Dept producer, Comm agreement, producer Select


Cluster  3  centroid
Room, route, Residence, conference, conference Room, Airport, floor, Residence Residence, Room floor, treaty Room


Cluster  4  centroid
check, schedule, confirm, list, Cheryl, Jake, travels, copy, bill, thought


Cluster  5  centroid
Blackberry, votes, START, women, parties, Obama, election, deal, developing, administration


Cluster  6  centroid
drafted, edits, comments, letter, drafted letter, Review, revised, revised drafted, Lissa, Jake




Выделяются кластера с темами про выборы; письма для помощников Хиллари Клинтон с какими-то указаниями;про расписание

Посмотрим на частотные n-граммы

In [4080]:
curLabel = 0
mailsLabeled = []
curMails = []

indexes = km.labels_.argsort()

for labelInd in indexes:
    label = km.labels_[labelInd]
    if label != curLabel:
        curLabel = label
        mailsLabeled.append(curMails)
        curMails = []
        
   
    curMails.append(labelInd)
mailsLabeled.append(curMails)
for i in range(n_clusters):
    print("Cluster ", i)
    print(", ".join([
            " ".join([stemToWord[v] for v in name.split("_")]) for name in countNGramms(2, mails, mailsLabeled[i])]))
    print()

Cluster  0
INFORMATION REDACTIONS, REDACTIONS FOIA, SUBJECT AGREEMENT, SENSITIVE INFORMATION, FOIA WAIVER, AGREEMENT SENSITIVE, BENGHAZI COMM, PRODUCED HOUSE, COMM SUBJECT, SELECT BENGHAZI

Cluster  1
State Dept, HOUSE SELECT, SELECT BENGHAZI, Date State, BENGHAZI COMM, SENSITIVE INFORMATION, REDACTIONS FOIA, COMM SUBJECT, SUBJECT AGREEMENT, PRODUCED HOUSE

Cluster  2
week June, meet week, behalf brother, Dougherty reporting, plan conf, Donilon told, Thank kind, kind offer, comment Washington, stars aligned

Cluster  3
advice watch, Chris hill, word advice, watch step, Morocco ground, forwarded Jake, Thanks send, reviewing edits, Thanks s000, anyone stature

Cluster  4
Secretary office, White house, state Department, units state, have been, Secretary state, Private residing, health care, conference Room, President Obama

Cluster  5
states Department, secretary office, unit states, having been, Private residents, Strategic Dialogue, Port Prince, Dialogue press, press Clips, office time


## Оценка

1) Цель: проверить, насколько хорошо связаны слова внутри кластеров

Задание: найдите лишнее слово.

Предлагается 5 слов, описывающих кластер, + слово из другого кластера
Для каждого кластера, получаем величину 1-"Кол-во ошибок/кол-во асессоров"

UPD: Опрос показал, что лучше искать несколько лишних слов.



Проблема с интрепретацией самого большого кластера. 

2) Цель: проверить, насколько хорошо различаются слова из разных кластеров

Задание:  Даны n+1 слов из n  кластеров(при этом 2 слова из одного кластера). Найдите пару.

Для каждого кластера, получаем величину 1- "Кол-во ошибок/кол-во асессоров"

UPD: Лучше выбирать небольшое количество слов. Здесь стало показательным, что кластеризация плохая.


3) Цель: проверить, насколько хорошо связаны слова внутри кластеров

Задание: Дано описание кластера. Исключите лишние слова.


Итог: С 3 заданием сработало лучше всего. В целом кластеризация не очень удалась. 
Очертания некоторых тем выделяются.